# Cats and Dogs

In this kernel we will be training a CNN model in Keras to identify if the image have a cat or a dog. We have a dataset of 25000 images dogs and cats and we will use that to train our model for predicting results

We will create this whole Kernel in three basic steps:

* Importing, Preprocessing and Visualising the dataset
* Creating the CNN model
* Finally, Evaluating results and Making predictions

If you found this kernel useful please **UPVOTE**. 

So, let's get started

# Step 1 :
## Importing, Preprocessing and Visualising

In [ ]:
# Importing the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import keras
from keras import layers
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Dense, Dropout, Flatten
from keras.models import Sequential

Let's import our dataset. We will read images one by one from the **train_dir** and will store them in the **train_images** array. Each image is read in Grayscale and will be resized to (50, 50).

**train_labels** will store the labels for each of the images.

In [ ]:
# Importing and Processing the dataset

train_dir = '../input/train/'

train_images = []
train_labels = []

for img in tqdm(os.listdir(train_dir)):
    try:
        img_r = cv2.imread(os.path.join(train_dir, img), cv2.IMREAD_GRAYSCALE)
        train_images.append(np.array(cv2.resize(img_r, (50, 50), interpolation=cv2.INTER_CUBIC)))
        if 'dog' in img:
            train_labels.append(1)
        else:
            train_labels.append(0)
    except Exception as e:
        print('broken image')


Now, let's visualise the images in our dataset. As you can see in the bottom we have an image of a cute little kitty. Though, it's not very clear to us but model will not have much problems while learning from this image.

In [ ]:
# Visualising the image

plt.title(train_labels[0])
_ = plt.imshow(train_images[0])

Here, we have used **train_test_split** from **scikit-learn** to split our training images and labels into train and test data. Since, we have our test_size set to 0.2 our initial dataset will be split into 20000 training and 5000 test data elements.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [ ]:
print("Train Shape:" + str(x_train.shape))
print("Test Shape:" + str(x_test.shape))

In [ ]:
plt.title(y_train[0])
_ = plt.imshow(x_train[0])

# Step 2:
## Creating the CNN Model

Now, we will start to build our Convolutional Neural Network model. We will feed training images to our model and then it will be used to make predictions. This is a very basic model and a much better CNN architecture can be used to increase the accuracy.

To read more about CNNs you can refer to this [link](http://cs231n.github.io/convolutional-networks/)

In [ ]:
def baseline_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 1), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
model = baseline_model()

After creating the model we will compile it. Compiling a model means to "Configure the learning process". Here, we have defined our optimizer, loss and evaluation metrics.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
x_train = x_train.reshape(-1, 50, 50, 1)
x_test = x_test.reshape(-1, 50, 50, 1)

In [ ]:
history = model.fit(np.array(x_train), y_train, validation_data=(np.array(x_test), y_test), epochs=20, verbose=1)

We have trained our model on the training images and labels. The history object stores all the details in the process of training the model. So, let's move on to analysing our the process of training and the results we have got.

# Step 3:
## Evaluating results and Making Predictions

In [ ]:
hist = history.history

The plot below shows how the loss in both training(**x_train**) and validation(**x_test**) over the span of ten epochs. We can see how the decrease in the validation loss has slowed down while we approach the final epcohs.This may have happened because of a little bit of variance. You can observe similar trends in training and validation accuracy.

In [ ]:
plt.plot(hist['loss'], 'green', label='Training Loss')
plt.plot(hist['val_loss'], 'blue', label='Validation Loss')
_ = plt.legend()

The plot below shows the change in accuracy in all the epochs. From the plot we can clearly see that there is a slight slowdown in the increase in validation accuracy. This is because of the reasons I mentioned previously. In our model, this problem is very much under the control but sometimes it can be very visible. In such cases there are many ways to solve it, just do a quick google search to learn more about it.

In [ ]:
plt.plot(hist['acc'], 'green', label='Training Accuracy')
plt.plot(hist['val_acc'], 'blue', label='Validation Accuracy')
_ = plt.legend()

Finally we will import the test data to make predictions. Note that this is not the same test data set which we created by splitting our training set. That dataset is usually referred as **validation** set. We use validation set to make sure that our model is not overfitting the training data.

In [ ]:
test_dir = '../input/test/'
test_images = []

for img in tqdm(os.listdir(test_dir)):
    try:
        img_r = cv2.imread(os.path.join(test_dir, img), cv2.IMREAD_GRAYSCALE)
        test_images.append(np.array(cv2.resize(img_r, (50, 50), interpolation=cv2.INTER_CUBIC)))
    except Exception as e:
        print('broken image')

In [ ]:
_ = plt.imshow(test_images[0])

After analysing the model we will make the final predictions and will create the submission file

In [ ]:
test_images = np.array(test_images)
test_images = test_images.reshape(-1, 50, 50, 1)
predictions = model.predict(test_images)

In [ ]:
predictions.shape

In [ ]:
counter = range(1, len(test_images) + 1)
solution = pd.DataFrame({"id": counter, "label":list(predictions)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("dogsVScats.csv", index = False)
